# 라이브러리 로딩

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT

import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

# 데이터 불러오기

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')

# 전처리

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin1        8590 non-null   object 
 4   Cabin2        8590 non-null   float64
 5   Combi         8590 non-null   object 
 6   Cabin3        8590 non-null   object 
 7   Cabin         8590 non-null   object 
 8   Destination   8693 non-null   object 
 9   Age           8693 non-null   int64  
 10  VIP           8693 non-null   bool   
 11  RoomService   8693 non-null   int64  
 12  FoodCourt     8693 non-null   int64  
 13  ShoppingMall  8693 non-null   int64  
 14  Spa           8693 non-null   int64  
 15  VRDeck        8693 non-null   int64  
 16  Name          8493 non-null   object 
 17  Transported   8693 non-null   bool   
dtypes: bool(3), float64(1), int6

## 필요없는 features 제거

In [4]:
# 필요없는 features 제거
data.drop(['PassengerId', 'Cabin', 'Cabin2', 'Name', 'Combi'], axis=1, inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8693 non-null   object
 1   CryoSleep     8693 non-null   bool  
 2   Cabin1        8590 non-null   object
 3   Cabin3        8590 non-null   object
 4   Destination   8693 non-null   object
 5   Age           8693 non-null   int64 
 6   VIP           8693 non-null   bool  
 7   RoomService   8693 non-null   int64 
 8   FoodCourt     8693 non-null   int64 
 9   ShoppingMall  8693 non-null   int64 
 10  Spa           8693 non-null   int64 
 11  VRDeck        8693 non-null   int64 
 12  Transported   8693 non-null   bool  
dtypes: bool(3), int64(6), object(4)
memory usage: 704.7+ KB


## 처리하기 힘든 결측값 제거

In [6]:
data.isna().sum()

HomePlanet        0
CryoSleep         0
Cabin1          103
Cabin3          103
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [7]:
# 결측값들 제거(Cabin)
data.dropna(axis=0, inplace=True)

## Boolean 캐스팅

In [8]:
# Cabin3의 값을 변환
data['Cabin3'].replace({'P': True,'S': False}, inplace=True)
data['Cabin3'] = data['Cabin3'].astype(bool)

## 원핫인코딩

In [9]:
# 원핫인코딩
train_encoding = pd.get_dummies(data['HomePlanet'])
data=data.drop('HomePlanet',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Destination'])
data=data.drop('Destination',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Cabin1'])
data=data.drop('Cabin1',axis=1)
data = data.join(train_encoding)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8590 entries, 0 to 8692
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   CryoSleep      8590 non-null   bool 
 1   Cabin3         8590 non-null   bool 
 2   Age            8590 non-null   int64
 3   VIP            8590 non-null   bool 
 4   RoomService    8590 non-null   int64
 5   FoodCourt      8590 non-null   int64
 6   ShoppingMall   8590 non-null   int64
 7   Spa            8590 non-null   int64
 8   VRDeck         8590 non-null   int64
 9   Transported    8590 non-null   bool 
 10  Earth          8590 non-null   uint8
 11  Europa         8590 non-null   uint8
 12  Mars           8590 non-null   uint8
 13  55 Cancri e    8590 non-null   uint8
 14  PSO J318.5-22  8590 non-null   uint8
 15  TRAPPIST-1e    8590 non-null   uint8
 16  A              8590 non-null   uint8
 17  B              8590 non-null   uint8
 18  C              8590 non-null   uint8
 19  D     

## 스케일링

In [11]:
# 스케일링
col = ['Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [12]:
data_scaled(data, col)

,CryoSleep,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,PSO J318.5-22,TRAPPIST-1e,A,B,C,D,E,F,G,T
0,False,True,0.712274,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,False,...,0,1,0,1,0,0,0,0,0,0
1,False,False,-0.332624,False,-0.168530,-0.275148,-0.241196,0.220460,-0.224520,True,...,0,1,0,0,0,0,0,1,0,0
2,False,False,2.035811,True,-0.268567,1.959032,-0.282832,5.745469,-0.220106,False,...,0,1,1,0,0,0,0,0,0,0
3,False,False,0.294315,False,-0.333743,0.522818,0.335048,2.711463,-0.092988,False,...,0,1,1,0,0,0,0,0,0,0
4,False,False,-0.889902,False,0.125518,-0.236941,-0.031350,0.234796,-0.261596,True,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,True,0.851594,True,-0.333743,3.990274,-0.282832,1.200732,-0.198037,False,...,0,0,1,0,0,0,0,0,0,0
8689,True,False,-0.750583,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,False,...,1,0,0,0,0,0,0,0,1,0
8690,False,False,-0.193304,False,-0.333743,-0.280785,2.834877,-0.270573,-0.263361,True,...,0,1,0,0,0,0,0,0,1,0
8691,False,False,0.224655,False,-0.333743,0.376253,-0.282832,0.044835,2.592370,False,...,0,0,0,0,0,0,1,0,0,0


In [13]:
data.columns

Index(['CryoSleep', 'Cabin3', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'Earth', 'Europa',
       'Mars', '55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'T'],
      dtype='object')

# 데이터셋 분리

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['Transported'], axis=1),
                                                    data.Transported, random_state=109)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

# XGBoost

In [15]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4)}

In [16]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def bin_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate'],
                            gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = KFold(n_splits=3)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [17]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=bin_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.88826	validation_1-auc:0.87453                                                                  
[1]	validation_0-auc:0.90424	validation_1-auc:0.88421                                                                  
[2]	validation_0-auc:0.91625	validation_1-auc:0.88771                                                                  
[3]	validation_0-auc:0.92169	validation_1-auc:0.88994                                                                  
[4]	validation_0-auc:0.92532	validation_1-auc:0.88905                                                                  
[5]	validation_0-auc:0.92815	validation_1-auc:0.88807                                                                  
[6]	validation_0-auc:0.93043	validation_1-auc:0.88972                                                                  
[7]	validation_0-auc:0.93155	validation_1-auc:0.89078                                                                  
[8]	validation_0-auc:0.93388	validation_

[1]	validation_0-auc:0.88194	validation_1-auc:0.87545                                                                  
[2]	validation_0-auc:0.88411	validation_1-auc:0.87795                                                                  
[3]	validation_0-auc:0.88871	validation_1-auc:0.87948                                                                  
[4]	validation_0-auc:0.88961	validation_1-auc:0.87787                                                                  
[5]	validation_0-auc:0.89076	validation_1-auc:0.87783                                                                  
[6]	validation_0-auc:0.89503	validation_1-auc:0.88162                                                                  
[7]	validation_0-auc:0.89721	validation_1-auc:0.88362                                                                  
[8]	validation_0-auc:0.89983	validation_1-auc:0.88670                                                                  
[9]	validation_0-auc:0.90085	validation_

[45]	validation_0-auc:0.92528	validation_1-auc:0.87577                                                                 
[46]	validation_0-auc:0.92556	validation_1-auc:0.87565                                                                 
[47]	validation_0-auc:0.92596	validation_1-auc:0.87586                                                                 
[48]	validation_0-auc:0.92592	validation_1-auc:0.87583                                                                 
[49]	validation_0-auc:0.92636	validation_1-auc:0.87592                                                                 
[50]	validation_0-auc:0.92646	validation_1-auc:0.87581                                                                 
[51]	validation_0-auc:0.92659	validation_1-auc:0.87598                                                                 
[52]	validation_0-auc:0.92673	validation_1-auc:0.87609                                                                 
[53]	validation_0-auc:0.92693	validation

[0]	validation_0-auc:0.90439	validation_1-auc:0.87114                                                                  
[1]	validation_0-auc:0.92008	validation_1-auc:0.87844                                                                  
[2]	validation_0-auc:0.93534	validation_1-auc:0.87840                                                                  
[3]	validation_0-auc:0.94305	validation_1-auc:0.88074                                                                  
[4]	validation_0-auc:0.94686	validation_1-auc:0.88128                                                                  
[5]	validation_0-auc:0.95055	validation_1-auc:0.88386                                                                  
[6]	validation_0-auc:0.95249	validation_1-auc:0.88476                                                                  
[7]	validation_0-auc:0.95377	validation_1-auc:0.88535                                                                  
[8]	validation_0-auc:0.95724	validation_

[41]	validation_0-auc:0.98279	validation_1-auc:0.87295                                                                 
[42]	validation_0-auc:0.98306	validation_1-auc:0.87218                                                                 
[43]	validation_0-auc:0.98311	validation_1-auc:0.87193                                                                 
[44]	validation_0-auc:0.98329	validation_1-auc:0.87225                                                                 
[45]	validation_0-auc:0.98348	validation_1-auc:0.87255                                                                 
[46]	validation_0-auc:0.98396	validation_1-auc:0.87231                                                                 
[47]	validation_0-auc:0.98418	validation_1-auc:0.87218                                                                 
[0]	validation_0-auc:0.86179	validation_1-auc:0.86957                                                                  
[1]	validation_0-auc:0.86730	validation_

...

[37]	validation_0-auc:0.91879	validation_1-auc:0.89079                                                                 
[38]	validation_0-auc:0.91903	validation_1-auc:0.89089                                                                 
[39]	validation_0-auc:0.91903	validation_1-auc:0.89089                                                                 
[40]	validation_0-auc:0.91903	validation_1-auc:0.89089                                                                 
[41]	validation_0-auc:0.91917	validation_1-auc:0.89086                                                                 
[42]	validation_0-auc:0.92044	validation_1-auc:0.89125                                                                 
[43]	validation_0-auc:0.92120	validation_1-auc:0.89165                                                                 
[44]	validation_0-auc:0.92202	validation_1-auc:0.89251                                                                 
[45]	validation_0-auc:0.92202	validation

[39]	validation_0-auc:0.92246	validation_1-auc:0.88476                                                                 
[40]	validation_0-auc:0.92246	validation_1-auc:0.88476                                                                 
[41]	validation_0-auc:0.92246	validation_1-auc:0.88476                                                                 
[42]	validation_0-auc:0.92246	validation_1-auc:0.88476                                                                 
[43]	validation_0-auc:0.92246	validation_1-auc:0.88476                                                                 
[44]	validation_0-auc:0.92246	validation_1-auc:0.88476                                                                 
[0]	validation_0-auc:0.84749	validation_1-auc:0.83222                                                                  
[1]	validation_0-auc:0.87734	validation_1-auc:0.86799                                                                  
[2]	validation_0-auc:0.88474	validation_

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[45]	validation_0-auc:0.95247	validation_1-auc:0.87733                                                                 
[46]	validation_0-auc:0.95247	validation_1-auc:0.87733                                                                 
[47]	validation_0-auc:0.95297	validation_1-auc:0.87739                                                                 
[48]	validation_0-auc:0.95297	validation_1-auc:0.87739                                                                 
[49]	validation_0-auc:0.95297	validation_1-auc:0.87739                                                                 
[50]	validation_0-auc:0.95301	validation_1-auc:0.87745                                                                 
[51]	validation_0-auc:0.95301	validation_1-auc:0.87745                                                                 
[52]	validation_0-auc:0.95301	validation_1-auc:0.87745                                                                 
[53]	validation_0-auc:0.95370	validation

[83]	validation_0-auc:0.96295	validation_1-auc:0.88003                                                                 
[84]	validation_0-auc:0.96295	validation_1-auc:0.88003                                                                 
[85]	validation_0-auc:0.96295	validation_1-auc:0.88003                                                                 
[86]	validation_0-auc:0.96295	validation_1-auc:0.88003                                                                 
[87]	validation_0-auc:0.96295	validation_1-auc:0.88003                                                                 
[88]	validation_0-auc:0.96295	validation_1-auc:0.88003                                                                 
[89]	validation_0-auc:0.96295	validation_1-auc:0.88003                                                                 
[90]	validation_0-auc:0.96295	validation_1-auc:0.88003                                                                 
[91]	validation_0-auc:0.96295	validation

In [18]:
# 평가용 함수
def  get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
#     roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, \
    재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [26]:
xgbo = xgb.XGBClassifier(colsample_bytree=0.9227415127512297, gamma=3.9860828876917274,
                         learning_rate=0.06795599597693397,
                         max_depth=9, min_child_weight=8, random_state=109)
xgbo.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.9227415127512297, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=3.9860828876917274, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.06795599597693397, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=9, max_leaves=0, min_child_weight=8, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

In [27]:
train_pred = xgbo.predict(X_train)
train_proba = xgbo.predict_proba(X_train)

test_pred = xgbo.predict(X_test)
test_proba = xgbo.predict_proba(X_test)

val_pred = xgbo.predict(X_val)
val_proba = xgbo.predict_proba(X_val)

In [28]:
get_clf_eval(y_train, train_pred, train_proba)

오차 행렬
[[1976  430]
 [ 344 2081]]
정확도: 0.8398, 정밀도: 0.8288,     재현율: 0.8581, F1: 0.8432


In [29]:
get_clf_eval(y_test, test_pred, test_proba)

오차 행렬
[[838 229]
 [212 869]]
정확도: 0.7947, 정밀도: 0.7914,     재현율: 0.8039, F1: 0.7976


In [30]:
get_clf_eval(y_val, val_pred, val_proba)

오차 행렬
[[601 183]
 [139 688]]
정확도: 0.8001, 정밀도: 0.7899,     재현율: 0.8319, F1: 0.8104


In [33]:
fi = pd.DataFrame(xgbo.feature_importances_, index=data.drop('Transported', axis=1).columns)

In [34]:
fi.to_csv('fi_3.csv')